In [12]:
import napari
from napari.layers import Points
import numpy as np
from napari_clusters_plotter._new_plotter_widget import _export_cluster_to_layer, PlotterWidget

In [9]:
viewer = napari.Viewer(ndisplay=3)

In [13]:
def create_multi_point_layer(n_samples: int = 100):
    import pandas as pd

    loc = 5
    n_timeframes = 5
    frame = np.arange(n_timeframes).repeat(n_samples // n_timeframes)
    # make some random points with random  features
    points = np.random.random((n_samples, 4))
    points2 = np.random.random((n_samples - 1, 4))

    points[:, 0] = frame
    points2[:, 0] = frame[:-1]

    features = pd.DataFrame(
        {
            "frame": frame,
            "feature1": np.random.normal(size=n_samples, loc=loc),
            "feature2": np.random.normal(size=n_samples, loc=loc),
            "feature3": np.random.normal(size=n_samples, loc=loc),
            "feature4": np.random.normal(size=n_samples, loc=loc),
        }
    )

    features2 = pd.DataFrame(
        {
            "frame": frame[:-1],
            "feature2": np.random.normal(size=n_samples - 1, loc=-loc),
            "feature3": np.random.normal(size=n_samples - 1, loc=-loc),
            "feature4": np.random.normal(size=n_samples - 1, loc=-loc),
        }
    )

    layer = Points(
        points, features=features, size=0.1, blending="translucent_no_depth"
    )
    layer2 = Points(
        points2,
        features=features2,
        size=0.1,
        translate=(0, 0, 2),
        blending="translucent_no_depth",
    )

    return layer, layer2


def create_multi_tracks_layer(n_samples: int = 100):
    from napari.layers import Tracks

    points1, points2 = create_multi_point_layer(n_samples=n_samples)

    tracks1 = points1.data.copy()
    tracks2 = points2.data.copy()

    # insert empty track id column
    tracks1 = np.insert(tracks1, 0, 0, axis=1)
    tracks2 = np.insert(tracks2, 0, 0, axis=1)

    for t in range(int(points1.data[:, 0].max() + 1)):
        # set the track id for each point
        tracks1[tracks1[:, 1] == t, 0] = np.arange(
            len(tracks1[tracks1[:, 1] == t]), dtype=int
        )

    for t in range(int(points2.data[:, 0].max() + 1)):
        # set the track id for each point
        tracks2[tracks2[:, 1] == t, 0] = np.arange(
            len(tracks2[tracks2[:, 1] == t]), dtype=int
        )

    tracks1 = Tracks(tracks1, features=points1.features, name="tracks1")
    tracks2 = Tracks(
        tracks2, features=points2.features, name="tracks2", translate=(0, 0, 2)
    )

    return tracks1, tracks2

In [14]:
layer1, layer2 = create_multi_tracks_layer()
viewer.add_layer(layer1)
viewer.add_layer(layer2)
viewer.layers.select_all()

In [15]:
n_layers = len(viewer.layers)

widget = PlotterWidget(viewer)
viewer.window.add_dock_widget(widget, area="right")

c:\Users\Johan\mambaforge\envs\clusters-plotter\Lib\site-packages\biaplotter\colormap.py:34: UserWarning: Categorical colormap detected. Setting categorical=True. If the colormap is continuous, set categorical=False explicitly.
  warnings.warn(


In [ ]:
for layer in viewer.layers:
    if type(layer) in widget.input_layer_types:
        features = layer.features
        features['MANUAL_CLUSTER_ID'] = np.random.randint(low=0, high=2, size=len(features))
        layer.features = features

widget.plot_needs_update.emit()

In [ ]:
widget._on_export_clusters()

In [20]:
viewer.layers[0].data.shape

(100, 5)